In [3]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time
from API_keys import api_key, api_key_secret, access_token, access_token_secret
from datetime import datetime
from scraping_tweets import scraptweets

In [ ]:
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# search_words = "#hongkong OR #hkprotests"
# date_since = "2019-11-03"
# numTweets = 20
# numRuns = 6
# # Call the function scraptweets

# scraptweets(search_words, date_since, numTweets, numRuns)

API.search(q[, geocode][, lang][, locale][, result_type][, count][, until][, since_id][, max_id][, include_entities])

In [ ]:
db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                    'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                    'retweetcount', 'text', 'hashtags'])

In [ ]:
search_words = "#coffee or #Coffee"
date_since = "2017-09-01"
numTweets = 1000
numRuns = 20

In [ ]:
print(search_words)
print(numTweets)
print(date_since)

In [ ]:
for i in range(0, numRuns):
    # We will time how long it takes to scrape tweets for each run:
    start_run = time.time()

    # Collect tweets using the Cursor object
    # .Cursor() returns an object that you can iterate or loop over to access the data collected.
    # Each item in the iterator has various attributes that you can access to get information about each tweet
    tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)

    # Store these tweets into a python list
    tweet_list = [tweet for tweet in tweets]

    # Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet

    # Begin scraping the tweets individually:
    noTweets = 0

    for tweet in tweet_list:

        # Pull the values
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            text = tweet.full_text

        # Add the 11 variables to the empty list - ith_tweet:
        ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                     usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

        # Append to dataframe - db_tweets
        db_tweets.loc[len(db_tweets)] = ith_tweet

        # increase counter - noTweets  
        noTweets += 1

    # Run ended:
    end_run = time.time()
    duration_run = round(end_run-start_run, 2)

    print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
    print('time take for {} run to complete is {}'.format(i, duration_run))

    time.sleep(30) #30second sleep time


In [ ]:
db_tweets

In [ ]:
# Once all runs have completed, save them to a single csv file:    
# Obtain timestamp in a readable format:

to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

# Define working path and filename
path = os.getcwd()
filename = to_csv_timestamp + '_coffee_tweets.csv'

# Store dataframe in csv with creation date timestamp
db_tweets.to_csv(filename, index = False)

print('Scraping has completed!')

In [ ]:
import os
os.listdir()